In [13]:
import os
import numpy as np
import string
from nltk.corpus import stopwords
from tqdm import tqdm
import pandas as pd

In [27]:
train_path = "C:/Users/Sree/Downloads/aclImdb/train/" # source data
test_path = "C:/Users/Sree/Downloads/aclImdb/test/" # test data for grade evaluation. 

In [15]:
def remove_stopwords(message):
    message = [c for c in message.split() if c.lower() not in stopwords.words('english')]
    return message


In [28]:
indices2 = []
text2 = []
rating2 = []
i = 0
for filename in tqdm(os.listdir(test_path+"pos")):
    data = open(test_path+"pos/"+filename, 'r' , encoding="ISO-8859-1").read()
    message = [c for c in data if c not in string.punctuation]
    message = ''.join(message)
    indices2.append(i)
    text2.append(message)
    rating2.append("1")
    i = i + 1

100%|██████████| 997/997 [00:09<00:00, 101.16it/s]


In [16]:
indices = []
text = []
rating = []
i = 0
for filename in tqdm(os.listdir(train_path+"pos")):
    data = open(train_path+"pos/"+filename, 'r' , encoding="ISO-8859-1").read()
    message = [c for c in data if c not in string.punctuation]
    message = ''.join(message)
    indices.append(i)
    text.append(message)
    rating.append("1")
    i = i + 1

100%|██████████| 1005/1005 [00:00<00:00, 2659.46it/s]


In [29]:
for filename in tqdm(os.listdir(test_path+"neg")):
    data = open(test_path+"neg/"+filename, 'r' , encoding="ISO-8859-1").read()
    message = [c for c in data if c not in string.punctuation]
    message = ''.join(message)
    indices2.append(i)
    text2.append(message)
    rating2.append("0")
    i = i + 1

100%|██████████| 1000/1000 [00:08<00:00, 123.68it/s]


In [17]:
for filename in tqdm(os.listdir(train_path+"neg")):
    data = open(train_path+"neg/"+filename, 'r' , encoding="ISO-8859-1").read()
    message = [c for c in data if c not in string.punctuation]
    message = ''.join(message)
    indices.append(i)
    text.append(message)
    rating.append("0")
    i = i + 1

100%|██████████| 1001/1001 [00:00<00:00, 2159.54it/s]


In [18]:
reviews = pd.DataFrame(data=(text),columns = ['reviews'])

In [30]:
reviews2 = pd.DataFrame(data=(text2),columns = ['reviews'])

In [19]:
reviews["rating"] = rating

In [32]:
reviews2["rating"] = rating2

In [20]:
reviews.describe()

,reviews,rating
count,2006,2006
unique,2000,2
top,The story and the show were good but it was re...,1
freq,2,1005


In [33]:
reviews2.describe()

,reviews,rating
count,1997,1997
unique,1995,2
top,Sigh Im baffled when I see a short like this g...,0
freq,2,1000


In [21]:
reviews.head()

,reviews,rating
0,I liked the film Some of the action scenes wer...,1
1,Somewhat funny and wellpaced action thriller t...,1
2,Just two commentsSEVEN years apart Hardly evid...,1
3,Another Aussie masterpiece this delves into th...,1
4,After a brief prologue showing a masked man st...,1


In [34]:
reviews2.head()

,reviews,rating
0,I went and saw this movie last night after bei...,1
1,The finest short Ive ever seen Some commentato...,1
2,This is a very very odd filmone that is so odd...,1
3,Although Bullet In The Brain is without questi...,1
4,means take up and read which is precisely what...,1


In [22]:
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
bow_transformer = CountVectorizer(analyzer=remove_stopwords).fit(reviews['reviews'])

In [35]:
bow_transformer2 = CountVectorizer(analyzer=remove_stopwords).fit(reviews2['reviews'])

In [24]:
messages_bow = bow_transformer.transform(reviews['reviews'])

In [36]:
messages_bow2 = bow_transformer.transform(reviews2['reviews'])

In [40]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow)

In [41]:
tfidf_transformer2 = TfidfTransformer().fit(messages_bow2)

In [42]:
messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)

(2006, 32155)


In [43]:
messages_tfidf2 = tfidf_transformer.transform(messages_bow2)
print(messages_tfidf2.shape)

(1997, 32155)


In [44]:
from sklearn.naive_bayes import MultinomialNB

In [45]:
mnb = MultinomialNB()

In [46]:
mnb.fit(messages_tfidf,reviews['rating'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [47]:
prediction = mnb.predict(messages_tfidf2)

In [48]:
from sklearn.metrics import classification_report

In [49]:
print(classification_report(reviews2['rating'],prediction))

              precision    recall  f1-score   support

           0       0.77      0.91      0.83      1000
           1       0.89      0.72      0.80       997

    accuracy                           0.82      1997
   macro avg       0.83      0.82      0.81      1997
weighted avg       0.83      0.82      0.81      1997



In [50]:
from sklearn.metrics import confusion_matrix

In [52]:
print(confusion_matrix(reviews2['rating'],prediction))

[[909  91]
 [276 721]]
